In [9]:
%matplotlib inline
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam


In [10]:
PATH = os.getcwd()
# Define data path
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)

num_channel=1
num_epoch=10

# Define the number of classes
num_classes = 3

img_data_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
        
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img)
        if input_img is not None:
                    	 	#print input_img
            input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY) 
            input_img_resize=cv2.resize(input_img,(299,299))
            img_data_list.append(input_img_resize)
               
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255


Loaded the images of dataset-Note

Loaded the images of dataset-Report

Loaded the images of dataset-Scientific



In [11]:
if num_channel==1:
	if K.image_dim_ordering()=='th':
		img_data= np.expand_dims(img_data, axis=1) 
		print (img_data.shape)
	else:
		img_data= np.expand_dims(img_data, axis=4) 
		
else:
	if K.image_dim_ordering()=='th':
		img_data=np.rollaxis(img_data,3,1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  


In [12]:
# Assigning Labels
USE_SKLEARN_PREPROCESSING=False

if USE_SKLEARN_PREPROCESSING:
	# using sklearn for preprocessing
	from sklearn import preprocessing
	
	def image_to_feature_vector(image, size=(128, 128)):
		# resize the image to a fixed size, then flatten the image into
		# a list of raw pixel intensities
		return cv2.resize(image, size).flatten()
	
	img_data_list=[]
	for dataset in data_dir_list:
		img_list=os.listdir(data_path+'/'+ dataset)
		print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
		for img in img_list:
			input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
			input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
			input_img_flatten=image_to_feature_vector(input_img,(128,128))
			img_data_list.append(input_img_flatten)
	
	img_data = np.array(img_data_list)
	img_data = img_data.astype('float32')
	print (img_data.shape)
	img_data_scaled = preprocessing.scale(img_data)
	print (img_data_scaled.shape)
	
	print (np.mean(img_data_scaled))
	print (np.std(img_data_scaled))
	
	print (img_data_scaled.mean(axis=0))
	print (img_data_scaled.std(axis=0))
	
	if K.image_dim_ordering()=='th':
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],num_channel,img_rows,img_cols)
		print (img_data_scaled.shape)
		
	else:
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],img_rows,img_cols,num_channel)
		print (img_data_scaled.shape)
	
	
	if K.image_dim_ordering()=='th':
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],num_channel,img_rows,img_cols)
		print (img_data_scaled.shape)
		
	else:
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],img_rows,img_cols,num_channel)
		print (img_data_scaled.shape)

if USE_SKLEARN_PREPROCESSING:
	img_data=img_data_scaled

In [13]:
num_classes = 3

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:202]=0
labels[202:464]=1
labels[464:]=2

names = ['Note','Scientific','Report']

In [14]:
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

input_shape=img_data[0].shape

In [7]:
#Defining Model
model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(299,299,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

# Viewing model_configuration

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 299, 299, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 297, 297, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 297, 297, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 146, 146, 64)      18496     
__________

In [8]:
# Training
hist = model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))


# Evaluating the model 

score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

# Confusion Matrix

from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

target_names = ['class 0(Note)', 'class 1(scientific)','class 2(report)']

print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

Train on 581 samples, validate on 146 samples
Epoch 1/10
581/581 [==============================] - 209s 360ms/step - loss: 3.5945 - acc: 0.4182 - val_loss: 1.0481 - val_acc: 0.4726
Epoch 2/10
581/581 [==============================] - 207s 356ms/step - loss: 0.9539 - acc: 0.5404 - val_loss: 0.8784 - val_acc: 0.5822
Epoch 3/10
581/581 [==============================] - 209s 360ms/step - loss: 0.7875 - acc: 0.6506 - val_loss: 0.9533 - val_acc: 0.4658
Epoch 4/10
581/581 [==============================] - 206s 355ms/step - loss: 0.7067 - acc: 0.7212 - val_loss: 0.6038 - val_acc: 0.7260
Epoch 5/10
581/581 [==============================] - 206s 355ms/step - loss: 0.6131 - acc: 0.7556 - val_loss: 0.6486 - val_acc: 0.7055
Epoch 6/10
581/581 [==============================] - 207s 357ms/step - loss: 0.5138 - acc: 0.8210 - val_loss: 0.6457 - val_acc: 0.7192
Epoch 7/10
581/581 [==============================] - 207s 356ms/step - loss: 0.3799 - acc: 0.8623 - val_loss: 0.6923 - val_acc: 0.7123
Ep

In [ ]:
# With 10 epochs, we get the Training accuracy=93.63 
# We can tune the model with the variation of number of epochs and increasing the data.
# Large batch size may converge faster and give better results